# DEM Download

In [2]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import requests
from dotenv import load_dotenv
from tqdm.auto import tqdm

from deepfreezer.utils import ROOT_DIR

# Load environment variables from .env file
load_dotenv()

%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.constrained_layout.use"] = True

DOWNLOAD = False
EXTRACT = False
REPROJECT = False

In [3]:
def urls_from_file(path: str | Path) -> list[str]:
    with open(path) as file:
        return [line.replace("\n", "") for line in file]

In [4]:
def download_file(url: str, path: str | Path, **requests_kwargs) -> None:
    # path = Path(path)
    # Make the request and get the response
    response = requests.get(url, stream=True, **requests_kwargs)
    # Get file size from headers (in bytes)
    total_size = int(response.headers.get("content-length", 0))

    # Initialize progress bar
    pbar = tqdm(
        total=total_size,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        desc=f"Downloading {path.name}",
        leave=False,
    )

    # Open file for writing in binary mode
    with open(path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            # filter out keep-alive chunks
            if chunk:
                size = f.write(chunk)
                pbar.update(size)
    pbar.close()


In [5]:
def download_files(path: str | Path, **requests_kwargs) -> None:
    for url in tqdm(urls_from_file(path), unit="File", desc="Total progress"):
        file_name = url.split("/")[-1]
        file_path = edm_dir / file_name
        download_file(url, file_path, **requests_kwargs)
    print("Download completed.")


In [6]:
# Get EOC Geoservice Authentification
pwd = os.environ.get("EOC_GEO_PWD")
usr = os.environ.get("EOC_GEO_USR")

edm_dir = ROOT_DIR / "data/raw/TDM30_EDM"
edm_url_file = edm_dir / "TDM30_EDEM-url-list.txt"

# Download TDM30 EDM Tiles
if DOWNLOAD:
    download_files(edm_url_file, auth=(usr, pwd))

In [7]:
import zipfile

# Unzip files
if EXTRACT:
    for zip_file in edm_dir.glob("*.zip"):
        with zipfile.ZipFile(zip_file, "r") as zip_ref:
            zip_ref.extractall(edm_dir)
        print(f"Extracted: {zip_file.name}")

In [8]:
# Create the edm_tiffs directory if it doesn't exist
edm_tiffs_dir = edm_dir / "EDEM_EPSG4979_GeoTIFF"
edm_tiffs_dir.mkdir(exist_ok=True)

# Iterate through all EDEM folders in edm_dir
for folder in edm_dir.glob("TDM1_EDEM*/"):
    tif = list((folder / "EDEM").glob("*_W84.tif"))
    if not len(tif) == 1:
        raise ValueError(
            f"Multiple TIFF files found in {folder / 'EDEM'}, but only one was expected."
        )
    else:
        tif = tif[0]
    # Copy TIFF files to edm_tiffs directory
    destination = edm_tiffs_dir / tif.name
    destination.write_bytes(tif.read_bytes())
    print(f"Copied: {tif.name} to {edm_tiffs_dir}")


Copied: TDM1_EDEM_10_N45E008_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N45E011_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N46E006_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N45E009_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N45E010_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N45E012_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N46E007_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: TDM1_EDEM_10_N46E005_EDEM_W84.tif to /Users/lgentn/Repositories/glaciermap/data/raw/TDM30_EDM/EDEM_EPSG4979_GeoTIFF
Copied: 

Reproject all GeoTIFFs from EPSG:4979 to EPSG:4326

In [9]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


def warp_geotiff(input_path, output_path, dst_crs):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update(
            {"crs": dst_crs, "transform": transform, "width": width, "height": height}
        )

        with rasterio.open(output_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )


In [10]:
src_files = list(edm_tiffs_dir.glob("*_W84.tif"))
warped_dir = ROOT_DIR / "data/raw/TDM30_EDM/EDEM_EPSG4326_GeoTIFF"
warped_dir.mkdir(exist_ok=True)

if REPROJECT:
    for src_file in tqdm(src_files):
        dst_file = warped_dir / src_file.name
        warp_geotiff(src_file, dst_file, "EPSG:4326")